reference: http://trap.ncirl.ie/1845/1/briangibbons.pdf

# 1. Import Packages

In [1]:
from urllib.request import urlopen
import json
#from bs4 import BeautifulSoup
#from lxml import html
#import requests
import pandas as pd
#import numpy as np
from openpyxl import load_workbook
#import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

# 2. Fuction

In [2]:
def writing_in_excel(path, df, sheet_name):
    writer = pd.ExcelWriter(path, datetime_format='m/d/yyyy')
    df.to_excel(writer, sheet_name = sheet_name, index=False, freeze_panes = (0,1))
    writer.save()
    writer.close()

In [3]:
def writing_sheets_in_excel(path, df, sheet_name):
    book = load_workbook(path)
    writer = pd.ExcelWriter(path, engine = 'openpyxl', date_format='m/d/yyyy')
    writer.book = book
    df.to_excel(writer, sheet_name = sheet_name, index=False, freeze_panes = (0,1))
    writer.save()
    writer.close()

In [4]:
def drop_xy(df):
    # list comprehension of the cols that end with '_y'
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)
    
    #to_replace = [x for x in df if x.endswith('_x')]
    df.columns = [c.lower().replace('_x', '') for c in df.columns] 

In [5]:
def creating_excel_by_keys(path, html):
    path = path
    raw_html = urlopen(html)
    data = json.load(raw_html)
    keys = list(data.keys())
    i = 0
    while i < len(keys):
        try:
            name = keys[i]
            df1 = pd.DataFrame(data[keys[i]])
            writing_in_excel(path + name + '.xlsx', df1, name)
        except:
            pass
        i += 1


# Test if crawler works

In [6]:
# info about bootstrap (overall)
bootstrap_html = urlopen("https://fantasy.premierleague.com/api/bootstrap-static/")
bootstrap = json.load(bootstrap_html)

In [7]:
# info about one player

player_html = urlopen("https://fantasy.premierleague.com/api/element-summary/235/")
player_raw = json.load(player_html)

In [8]:
bootstrap.keys()
player_raw.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

dict_keys(['fixtures', 'history', 'history_past'])

In [17]:
# From Bootstrap

events = pd.DataFrame(bootstrap['events'])
#game_settings = pd.DataFrame(bootstrap['game_settings'])
phases =  pd.DataFrame(bootstrap['phases']) 
teams =  pd.DataFrame(bootstrap['teams']) 
#total_players = pd.DataFrame(bootstrap['total_players'])
elements =  pd.DataFrame(bootstrap['elements']) 
element_stats =  pd.DataFrame(bootstrap['element_stats']) 
element_types =  pd.DataFrame(bootstrap['element_types'])

In [20]:
##events
#phases
teams

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1230,1270,1150,1190,1280,1330
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,2,None,False,0,1040,1080,1030,1060,1030,1050
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,3,None,False,0,1090,1120,1030,1070,1110,1180
3,36,0,None,4,0,Brighton,0,0,0,BHA,3,None,False,0,1050,1120,1040,1160,1030,1130
4,90,0,None,5,0,Burnley,0,0,0,BUR,3,None,False,0,1050,1110,1060,1130,1050,1050
5,8,0,None,6,0,Chelsea,0,0,0,CHE,4,None,False,0,1270,1300,1200,1320,1310,1350
6,31,0,None,7,0,Crystal Palace,0,0,0,CRY,3,None,False,0,1110,1100,1060,1130,1040,1020
7,11,0,None,8,0,Everton,0,0,0,EVE,3,None,False,0,1100,1160,1110,1190,1090,1160
8,13,0,None,9,0,Leicester,0,0,0,LEI,4,None,False,0,1230,1280,1180,1240,1110,1190
9,14,0,None,10,0,Liverpool,0,0,0,LIV,5,None,False,0,1340,1360,1300,1330,1340,1370


In [21]:
len(player_raw)

3

In [11]:
# From Player_raw

fixtures = pd.DataFrame(player_raw['fixtures'])
history = pd.DataFrame(player_raw['history'])
history_past =  pd.DataFrame(player_raw['history_past']) 


In [15]:
select = 'D'

events.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\2019//events2019.xlsx')
game_settings.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\game_settings2019.xlsx')
phases.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\phases2019.xlsx')
teams.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\teams2019.xlsx')
elements.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\elements2019.xlsx')
element_stats.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\element_stats2019.xlsx')
element_types.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\element_types2019.xlsx')
fixtures.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\fixtures2019.xlsx')
history.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\history2019.xlsx')
history_past.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\history_past2019.xlsx')

NameError: name 'game_settings' is not defined

# Defining a scraper

In [18]:
def player_data_scraper():
    i=1
    frames = []

    
    while i < 600:
        try:
            player_html = urlopen("https://fantasy.premierleague.com/api/element-summary/" + str(i) + '/')
            #bootstrap_html = urlopen("https://fantasy.premierleague.com/api/bootstrap-static/")
            
            player_raw = json.load(player_html)
            #bootstrap_raw = json.load(bootstrap_html)
            
            player = player_raw['history']
            #bootstrap = bootstrap_raw['elements']
            player_df = pd.DataFrame(player)
            #bootstrap_df = pd.DataFrame(bootstrap)
            
            #player_df['position'] = bootstrap_df.loc[bootstrap_df['id'] == i, 'element_type']
            
            frames.append(player_df)
            result_df = pd.concat(frames)
            result_df = pd.DataFrame(result_df)
            print(i)
            
        except:
            #Write all of the numbers for which there was errors to a file
            errfile = open(player_error, "a")
            errfile.write(str(i) + "\n")
            pass
            
    
        print (i)
        i += 1
    return result_df


# Run this

In [19]:
# Paths
select = 'D'
player_path = select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\2019\\players2019.xlsx'
fixture_path = select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\2019\\fixtures2019.xlsx'
player_error = select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\2019\\error\\error2019.txt'
bootstrap_path = select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\2019\\bootstrap2019.xlsx'


In [20]:
result = player_data_scraper()
result2 = result.copy()
result2 = result2.rename(columns = {'element':'id'})

path = "https://fantasy.premierleague.com/api/bootstrap-static/"
raw_html = urlopen(path)
data = json.load(raw_html)
bootstrap_df = pd.DataFrame(data['elements'])
writing_in_excel(bootstrap_path, bootstrap_df, 'elements')

teams_path = select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\2019\\teams.xlsx'
teams_df = pd.DataFrame(data['teams'])
writing_in_excel(teams_path, teams_df, 'teams')


writing_in_excel(player_path, result2, 'player')


# merging the bootstrap and player data. However, player data will have the number of data of how manyever the gameweeks are. 
'''df1 = result.copy()
df2 = bootstrap_df.copy()
df3 = df1.merge(df2, on='id')
drop_xy(df3)

history_bootstrap_path = select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\history_bootstrap.xlsx'
writing_in_excel(history_bootstrap_path, df3, 'player_history')'''


1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
124
124
125
125
126
126
127
127
128
128
129
129
130
130
131
131
132
132
133
133
134
134
135
135
136
136
137
137
138
138
139
139
140
140
141
141
142
142
143
143
144
144
145
145
146
146
147
147
148
148
149
149
150
150
151
151
152
152


"df1 = result.copy()\ndf2 = bootstrap_df.copy()\ndf3 = df1.merge(df2, on='id')\ndrop_xy(df3)\n\nhistory_bootstrap_path = select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\history_bootstrap.xlsx'\nwriting_in_excel(history_bootstrap_path, df3, 'player_history')"

# Debugging
